# Interactive predictive model fitting for classifying case/control using pathway data. 

Quang Nguyen   
Last updated 2022-04-27

In [204]:
import numpy as np 
import pandas as pd
import sys
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score, brier_score_loss, roc_auc_score
from sklearn.inspection import permutation_importance
from skbio.stats.composition import clr, multiplicative_replacement
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import pickle
import timeit

In [205]:
sys.path.insert(1, '../python/')
from pred_eval import prior_preprocess

In [206]:
np.random.seed(160497)

In [229]:
"""
This function performs some apriori preprocessing based on data set required 
"""
def prior_preprocess(feat, meta, outcome_label, pos_class):
    X = feat.to_numpy()
    Y = meta.loc[:, outcome_label].to_numpy()
    Y = Y[~np.all(X == 0, axis = 1)]
    y = [1 if i == pos_class else 0 for i in Y]
    X = X[~np.all(X == 0, axis=1)]
    return(X, y)

"""
This function performs the clr transformation with imputation 
"""
def clr_transform(arr):
    # impute with multiplicative replacement
    arr = multiplicative_replacement(arr)
    # clr transformation 
    arr = clr(arr)
    return(arr)

"""
This function creates a scikit-learn pipeline 
Here we use a base default random forest classifier and calibrated it using 5-fold cross-validation. 
The remainder calibrated classifier is an ensemble of the individual classifiers. If the sample size is low, 
we're going to use sigmoid calibration approach. 
For preprocessing, we created a custom preprocessing function using the CLR transformation using scikit-bio
"""
def create_pipeline(clr_trans=True):
    rf = RandomForestClassifier(n_estimators=500, max_features='sqrt')
    estimators = [("calib_rf", CalibratedClassifierCV())]
    if clr_trans == True:
        transf = FunctionTransformer(clr_transform, validate=True)
        estimators.insert(0, ("clr_transformer", transf))
    pipe = Pipeline(estimators)
    pipe.set_params(calib_rf__base_estimator=rf, calib_rf__cv=5, calib_rf__ensemble=True, 
                    calib_rf__method='sigmoid')
    return(pipe)

In [230]:
feat = pd.read_csv("../data/pred_pathway_ibd_feat.csv", index_col=0)
lab = pd.read_csv("../data/pred_pathway_ibd_metadata.csv", index_col=0)

In [231]:
feat.head()

,UNMAPPED,UNINTEGRATED,PWY-6737: starch degradation V,PWY-1042: glycolysis IV (plant cytosol),PWY-5686: UMP biosynthesis,PWY-6163: chorismate biosynthesis from 3-dehydroquinate,PWY-6386: UDP-N-acetylmuramoyl-pentapeptide biosynthesis II (lysine-containing),ILEUSYN-PWY: L-isoleucine biosynthesis I (from threonine),PWY-7111: pyruvate fermentation to isobutanol (engineered),VALSYN-PWY: L-valine biosynthesis,...,PWY-6148: tetrahydromethanopterin biosynthesis,P261-PWY: coenzyme M biosynthesis I,PWY-5079: L-phenylalanine degradation III,PWY-6957: mandelate degradation to acetyl-CoA,P101-PWY: ectoine biosynthesis,"PWY-6309: L-tryptophan degradation XI (mammalian, via kynurenine)",LEU-DEG2-PWY: L-leucine degradation I,PWY-7373: superpathway of demethylmenaquinol-6 biosynthesis II,PWY-7420: monoacylglycerol metabolism (yeast),ALL-CHORISMATE-PWY: superpathway of chorismate metabolism
SKST006_6_G102964,0.205630,0.750061,0.000681,0.000610,0.000576,0.000564,0.000551,0.000551,0.000551,0.000551,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SKST006_7_G102965,0.209946,0.742283,0.000639,0.000641,0.000574,0.000513,0.000592,0.000542,0.000542,0.000542,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SKST006_4_G102962,0.187781,0.766908,0.000628,0.000592,0.000553,0.000500,0.000542,0.000517,0.000517,0.000517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SKST006_5_G102963,0.218590,0.737545,0.000688,0.000685,0.000526,0.000483,0.000582,0.000519,0.000519,0.000519,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SKST006_2_G102960,0.232418,0.720824,0.000684,0.000644,0.000595,0.000596,0.000592,0.000604,0.000604,0.000604,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [232]:
lab.study_condition.value_counts()

IBD        2085
control     796
Name: study_condition, dtype: int64

In [233]:
np.array_equal(feat.index.values, lab.index.values)

True

In [234]:
X, y = prior_preprocess(feat, lab, "study_condition", "IBD")
pipe = create_pipeline(False)
pipe

Pipeline(steps=[('calib_rf',
                 CalibratedClassifierCV(base_estimator=RandomForestClassifier(max_features='sqrt',
                                                                              n_estimators=500),
                                        cv=5))])

In [222]:
start = timeit.default_timer()
scores = cross_validate(pipe, X, y, cv = 5, scoring=["balanced_accuracy", "neg_brier_score"])
stop = timeit.default_timer()
print('Time: ', stop - start)  

ValueError: Found input variables with inconsistent numbers of samples: [2870, 2881]

In [174]:
scores

{'fit_time': array([31.81757092, 27.94704795, 29.37093401, 37.14344192, 42.3275609 ]),
 'score_time': array([1.07529712, 0.71661019, 0.83429813, 0.83321214, 0.75956297]),
 'test_balanced_accuracy': array([0.5, 0.5, 0.5, 0.5, 0.5]),
 'test_neg_brier_score': array([-0.19708296, -0.20105617, -0.20265902, -0.2004177 , -0.18731096])}

In [239]:
pipe2 = make_pipeline(FunctionTransformer(clr_transform, validate=True), 
                      LogisticRegression(penalty='elasticnet', C = 0.1, l1_ratio = 0.5, 
                                        solver = 'saga'))
pipe2

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function clr_transform at 0x16b52af70>,
                                     validate=True)),
                ('logisticregression',
                 LogisticRegression(C=0.1, l1_ratio=0.5, penalty='elasticnet',
                                    solver='saga'))])

In [240]:
X = X
cross_validate(pipe2, X, y, cv = 5, scoring = "neg_brier_score")

/Users/quangnguyen/miniconda3/envs/pred_models/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/quangnguyen/miniconda3/envs/pred_models/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/quangnguyen/miniconda3/envs/pred_models/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/quangnguyen/miniconda3/envs/pred_models/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/quangnguyen/miniconda3/envs/pred_models/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which 

{'fit_time': array([1.28187799, 1.31699491, 1.28686929, 1.30763006, 1.2992928 ]),
 'score_time': array([0.00876093, 0.00545001, 0.00581193, 0.00697899, 0.00564003]),
 'test_score': array([-0.24725573, -0.21607755, -0.24980183, -0.22237276, -0.20137799])}

,test,fold,brier,roc_auc
0,test,1,0.197083,0.5
1,test,2,0.201056,0.5
2,test,3,0.202659,0.5
3,test,4,0.200418,0.5
4,test,5,0.187311,0.5
